In [ ]:
!pip install -q ultralytics torch torchvision roboflow

In [ ]:
import os
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="fXOGAkIOaCBMMrx0pgxg")
project = rf.workspace("my-personal-workspace-d84lu").project("yolo-fj4s3-gm7mf")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
params = {
    "imgsz": 512,
    "epochs": 50,
    "batch": 16,
    "optimizer": "SGD",
    "patience": 50,
    "workers": 8,
    "max_det": 300,
    "momentum": 0.937,
    "iou": 0.7
}

In [ ]:
# مسیر فایل yaml دیتاست
dataset_yaml = "YOLO-1/data.yaml"

In [ ]:
model = YOLO('yolov8s.pt')

results = model.train(
    data=dataset_yaml,
    imgsz=params["imgsz"],
    epochs=params["epochs"],
    batch=params["batch"],
    optimizer=params["optimizer"],
    patience=params["patience"],
    workers=params["workers"],
    max_det=params["max_det"],
    momentum=params["momentum"],
    iou=params["iou"],
    device=0  # اگر GPU داری، در غیر این صورت device="cpu"
)

In [ ]:
results.plot()
plt.show()

In [ ]:
from IPython.display import Image, display

# Display the training plot
display(Image(filename='runs/detect/train/results.png'))

In [ ]:
metrics = model.val(data=dataset_yaml, imgsz=params["imgsz"], batch=params["batch"])
print(metrics)

In [ ]:
from ultralytics.utils.plotting import plot_results
plot_results('runs/detect/train/results.csv')

In [ ]:
metrics.confusion_matrix.plot()

In [ ]:
results = model.predict('YOLO-1/test/images/y76_jpg.rf.9157d1b89ba0ac6b94cbc078bd180f5d.jpg', save=True, conf=0.25)

# Another Faster R-CNN Exp

In [ ]:
# (This is an example, your code might be slightly different)
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT')

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2) # num_classes = background + tumor


In [ ]:
import torch
from torchmetrics.detection import MeanAveragePrecision
import time

# --- Hyperparameters ---
NUM_EPOCHS = 30
LEARNING_RATE = 0.001
LR_STEP_SIZE = 10
LR_GAMMA = 0.1

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# --- Ensure model is on the correct device ---
# We assume 'model' is now a freshly initialized model in your environment
# after you re-ran your model definition cell.
model.to(device)

# --- Setup Optimizer and Scheduler ---
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP_SIZE, gamma=LR_GAMMA)

# Variables to track the best model
best_map = -1.0
best_model_weights = None

print("--- Starting Experiment 2 (Corrected): 30 Epochs with LR Scheduler ---")

# --- Main Training and Evaluation Loop ---
for epoch in range(NUM_EPOCHS):
    start_time = time.time()

    # --- Training Phase ---
    model.train()
    total_train_loss = 0
    for i, (images, targets) in enumerate(train_data_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_train_loss += losses.item()

    avg_train_loss = total_train_loss / len(train_data_loader)

    # Update the learning rate
    lr_scheduler.step()

    # --- Validation Phase ---
    model.eval()
    metric = MeanAveragePrecision(box_format='xyxy')
    with torch.no_grad():
        for images, targets in valid_data_loader:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            predictions = model(images)
            metric.update(predictions, targets)

    results = metric.compute()
    current_map = results['map'].item()

    # --- Epoch Summary ---
    epoch_time = time.time() - start_time
    print(
        f"Epoch {epoch+1}/{NUM_EPOCHS} | "
        f"Time: {epoch_time:.1f}s | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Validation mAP: {current_map:.4f}"
    )

    # Check if this is the best model so far
    if current_map > best_map:
        best_map = current_map
        best_model_weights = model.state_dict()
        print(f"*** New best model found! mAP: {best_map:.4f} at epoch {epoch+1} ***")

print("--- Finished Training ---")
print(f"Best Validation mAP achieved: {best_map:.4f}")

# Load the best performing model weights for future use
if best_model_weights:
    model.load_state_dict(best_model_weights)


In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import random

# --- Visualization Function (Corrected) ---
def visualize_predictions(model, data_loader, device, num_images=5):
    """
    Picks random images from a data loader, runs them through the model,
    and displays the ground truth and predicted bounding boxes.
    """
    model.eval()

    # Get a batch of images and targets
    images, targets = next(iter(data_loader))

    # Pick random images from the batch
    indices = random.sample(range(len(images)), k=min(num_images, len(images)))

    # Move images and model to the correct device
    images_for_model = [images[i].to(device) for i in indices]
    model.to(device)

    # Get predictions
    with torch.no_grad():
        predictions = model(images_for_model)

    # --- Plotting ---
    for i, (pred_dict, original_idx) in enumerate(zip(predictions, indices)):
        # Use the original, un-moved image for plotting
        image_tensor = images[original_idx]
        image_display = image_tensor.permute(1, 2, 0) # CHW -> HWC

        fig, ax = plt.subplots(1, figsize=(10, 8))
        ax.imshow(image_display)

        # --- Draw Ground Truth Boxes (in Green) ---
        true_boxes = targets[original_idx]['boxes']
        for box in true_boxes:
            xmin, ymin, xmax, ymax = box
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                 linewidth=2, edgecolor='g', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin, 'Ground Truth', color='g', bbox=dict(facecolor='white', alpha=0.5))

        # --- Draw Predicted Boxes (in Red) ---
        # *** THE FIX IS HERE: Move predictions to CPU ***
        pred_boxes = pred_dict['boxes'].cpu()
        pred_scores = pred_dict['scores'].cpu()

        # Filter out low-confidence predictions
        for box, score in zip(pred_boxes, pred_scores):
            if score > 0.5: # Confidence threshold
                xmin, ymin, xmax, ymax = box
                rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=2, edgecolor='r', facecolor='none')
                ax.add_patch(rect)
                ax.text(xmin, ymin, f'Pred: {score:.2f}', color='r', bbox=dict(facecolor='white', alpha=0.5))

        ax.set_title(f"Image {i+1}: Ground Truth (Green) vs. Prediction (Red)")
        plt.axis('off')
        plt.show()

# --- Run the visualization ---
print("--- Visualizing Model Predictions on Validation Set (Corrected) ---")
visualize_predictions(model, valid_data_loader, device, num_images=30)
